### In depth workflow
The code below provides a sample workflow for registering data which is particularly challenging, and may require some of the more detailed features of the rigidregistration package for successful processing.

These first few cells are prepretory: importing the necessary python libraries and functions, loading and pre-inspecting the data, and instantiating the rigidregistration 'imstack' object.  The bulk of the relevant functions here are defined as methods of the imstack object.  

In this example, data which is formatted as .tif files are loaded using the tifffile package.  For other file formats common to electron microscopy data (including, e.g., .dm3, .ser...) we recommend the hyperspy package for i/o handling.  See hyperspy.org.

If you find this code useful in your own research, please cite the associated publication:
"Image registration of low signal-to-noise cryo-STEM data", Ultramicroscopy (2018), DOI: 10.1016/j.ultramic.2018.04.008

In [30]:
# Import global libraries and functions
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import splitext, basename
from time import time
from tifffile import imread, imsave

# Import local libraries
import rigidregistration

%matplotlib

Using matplotlib backend: Qt5Agg


In [31]:
# Load data.  
# Final axis of stack variable should iterate over images.
# For best performance, data should be normalized between 0 and 1

f="sample_data/CdSe_NPs.tif"           # Filepath to data
stack=np.rollaxis(imread(f),0,3)       # Rearrange axes so final axis iterates over images
stack=stack[:,:,:]/float(2**16)        # Normalize data between 0 and 1
print("Analyzing {}.".format(f))

Analyzing sample_data/CdSe_NPs.tif.


In [32]:
# Inspect data in preparation for registration

for i in range(5,10):                      # Select which images from the stack to display
    fig,(ax1,ax2)=plt.subplots(1,2)
    ax1.matshow(stack[:,:,i],cmap='gray')
    ax2.matshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(stack[:,:,i])))),cmap='gray',vmin=np.average(np.log(np.abs(np.fft.fft2(stack[:,:,i]))))) 
    ax1.grid(False)
    ax2.grid(False)
    plt.show()

In [33]:
# Instantiate imstack object, and get all FFTs

s=rigidregistration.stackregistration.imstack(stack)    # Instantiage imstack object.
s.getFFTs()                                             # Calculate the FFTs of each image in the stack

### Fourier masking
Select a Fourier mask, defining which information will be weighted more or less to calculate the image shifts. 
There are two choices to be made: the cutoff frequency, and mask shape.  It is also possible to define a unique Fourier mask.

**1. The cutoff frequency**

In all cases, the parameter n controls the mask cutoff frequency; features smaller than ~n pixels will be ignored during image correlation.
For data with higher SNR, choosing a mask with n at the information limit is frequently sufficient.
For low-SNR data, choosing a mask with a cutoff frequency near the primary Bragg peaks is often preferable, as this heavily weights low frequency information to avoid unit-cell hops, but ideally contains just enough lattice information to 'lock-in' to the lattice.

**2. The mask shape**

Supported apodization functions for makeFourierMask() method are "bandpass", "lowpass", "hann", "hamming", "blackman", "gaussian", "none".
For lattices lacking high rotational symmetry, an anisotropic mask is generally preferable to avoid overweighting one lattice direction, discussed further below.
Details functional forms can be found in the source code.

**3. Defining a unique mask**

The imstack object contains several meshgrids which correspond to Fourier space coordinates, facilitating defining a unique Fourier mask.  See below.

In [34]:
# Explore mask shape options

# The displayed plots are:
# Left: The full FFT in black and white, with a colored, semitransparent overlay showing the mask
# Middle: The FFT multiplied by the weighting mask
# Right: The cross correlation of a selected pair of images, weighted by the chosen mask


masktypes=["bandpass","lowpass","hann","hamming","blackman","gaussian","none"]   # List of mask shapes
n=4                                                                              # Set cutoff frequency

i,j = 5,9                                    # Choose image pair to cross correlate
for masktype in masktypes:                   # Iterate over mask types
    s.makeFourierMask(mask=masktype,n=n)     # Set the selected Fourier mask
    s.show_Fourier_mask(i=i,j=j)             # Display results

In [11]:
# Vary n, the cutoff frequency

# In a typical workflow, this parameter is varied and the effect on the cross correlation is optimized

masktype="hann"

i,j = 5,9                                    # Choose image pair
for n in np.arange(2,12,2):                  # Select n values to test
    s.makeFourierMask(mask=masktype,n=n)     # Set the selected Fourier mask
    s.show_Fourier_mask(i=i,j=j)             # Display the results

In [8]:
# Elliptical Gaussian masks

# The makeFourierMask_eg() method creates an elliptical gaussian mask, with parameters n1, n2, and theta.
# n1, n2 define the cutoff frequencies along the two primary axes 
# theta defines the mask tilt, in degrees.

n1=4
n2=2
theta=20

s.makeFourierMask_eg(n1=n1,n2=n2,theta=np.radians(theta))   # Set the selected mask
s.show_Fourier_mask(i=i,j=j)                                # Display the results

In [8]:
# Vary elliptical Gaussian angle

# The three parameters (n1,n2,theta) can be varied and optimized.

n1=4
n2=2
thetas=[90,75,60,45,30,15,0]

i,j = 5,9
for theta in thetas:
    s.makeFourierMask_eg(n1=n1,n2=n2,theta=np.radians(theta))
    s.show_Fourier_mask(i=i,j=j)

In [29]:
# Define a unique mask

# There are three attributes of the imstack object which define Fourier space coordinates, and can be used to define 
# a Fourier mask.  kx and ky define rectangular Fourier coordinates, and kr defines polar Fourier coordinates.
# See the examples below.

circular_mask = np.abs(s.kr)<50                      # Make a circular mask
s.makeUserDefinedFourierMask(circular_mask)          # Set mask
s.show_Fourier_mask()

square_mask = (np.abs(s.kx)<50)*(np.abs(s.ky)<50)    # Make a square mask
s.makeUserDefinedFourierMask(square_mask)            # Set mask
s.show_Fourier_mask(i=i,j=j)

kmax=s.nx/4                                          # Set cutoff frequency
barlett_mask = (s.kr<kmax)*(1 - s.kr/kmax)           # Make bartlett mask
s.makeUserDefinedFourierMask(barlett_mask)           # Set mask
s.show_Fourier_mask(i=i,j=j)

### Calculate image shifts
Calculate the relative shifts between all pairs of images from their cross correlations.

Analytically, for two functions which are identical except for some shift, the shift is given by the maximum value of their cross correlation.
Here, the maximum of the cross correlation may be found in one of two ways: finding the brightest pixel, or fitting gaussian functions.  Which method is used is controlled by the findMaxima parameter.

** 1. Brightest pixel **

The shift is given directly by the position of the brightest pixel in the cross correlation.  The relative shifts are thus determined with resolution of 1 pixel - however, the final shifts which are applied to the images before averaging will be determined using all of the relative image shifts between all pairs of images, thus the final shifts may still be determined with subpixel resolution, with an accuracy that generally improves with the number of images in the stack.  This is the fastest approach.  This method is selected by setting findMaxima="pixel".

** 2. Gaussian fitting **

Fitting a continuous function to the maximum of the cross correlation is a simple way to find the shift between an image pair with subpixel resolution.  For images of atomic lattices, a Gaussian is a natural choice for a fitting function.

Separately, for images of crystal lattices, 'unit cell hop' errors can occur, wherein the calculated shift between a pair of images is incorrect by a multiple of the primitive lattice vectors.  When the real space sampling of the image is low enough that each atomic column is only a handful of pixels across, unit cell hops become increasingly common due to sampling error.  This can be largely handled by performing Gaussian fits to the regions near the brighest few pixels, and then finding the cross correlation maximum from these continuous fits.

This method is selected by setting findMaxima="gf".  Before running the Gaussian fitting method, three additional parameters for the fitting should be set by calling s.setGaussianFitParams().  These are:

  * sigma_guess: sets the initial guess for the standard deviation of the guassian fits, in pixels.  This may be estimated simply and quickly by observing the peak widths in the cross correlations or the width of atomic columns in the raw data.

  * window_radius: sets the size of the region about the brightest pixel which is used to fit a gaussian.  Should be set such that neighboring cross correlation peaks are excluded.  The window used is a square region of size length 2xwindow_radius+1.

  * num_peaks: sets how many of the brightest pixels to fit gaussians to. Typically 3-5 are sufficient to handle sampling problems.  

In [36]:
# Make elliptical Gaussian mask.
# The type and parameters of the Fourier mask are determined by varying parameters in the previous section.

n1=12
n2=9.106
theta=1.42

s.makeFourierMask_eg(n1=n1,n2=n2,theta=theta)
s.show_Fourier_mask(i=i,j=j)

In [12]:
findMaxima = 'gf'
s.setGaussianFitParams(num_peaks=5,sigma_guess=3,window_radius=4)

# Find all image shifts
t0=time()
s.findImageShifts(correlationType='cc',findMaxima=findMaxima)
t=time()-t0
print("Performed {} correlations in {} minutes {} seconds".format(s.nz*(s.nz-1)/2,int(t/60),t%60))

Correlating images 0 and 1
Correlating images 0 and 2
Correlating images 0 and 3
Correlating images 0 and 4
Correlating images 0 and 5
Correlating images 0 and 6
Correlating images 0 and 7
Correlating images 0 and 8
Correlating images 0 and 9
Correlating images 0 and 10
Correlating images 0 and 11
Correlating images 0 and 12
Correlating images 0 and 13
Correlating images 0 and 14
Correlating images 0 and 15
Correlating images 0 and 16
Correlating images 0 and 17
Correlating images 0 and 18
Correlating images 0 and 19
Correlating images 0 and 20
Correlating images 0 and 21
Correlating images 0 and 22
Correlating images 0 and 23
Correlating images 0 and 24
Correlating images 0 and 25
Correlating images 0 and 26
Correlating images 1 and 2
Correlating images 1 and 3
Correlating images 1 and 4
Correlating images 1 and 5
Correlating images 1 and 6
Correlating images 1 and 7
Correlating images 1 and 8
Correlating images 1 and 9
Correlating images 1 and 10
Correlating images 1 and 11
Correlati

Correlating images 16 and 18
Correlating images 16 and 19
Correlating images 16 and 20
Correlating images 16 and 21
Correlating images 16 and 22
Correlating images 16 and 23
Correlating images 16 and 24
Correlating images 16 and 25
Correlating images 16 and 26
Correlating images 17 and 18
Correlating images 17 and 19
Correlating images 17 and 20
Correlating images 17 and 21
Correlating images 17 and 22
Correlating images 17 and 23
Correlating images 17 and 24
Correlating images 17 and 25
Correlating images 17 and 26
Correlating images 18 and 19
Correlating images 18 and 20
Correlating images 18 and 21
Correlating images 18 and 22
Correlating images 18 and 23
Correlating images 18 and 24
Correlating images 18 and 25
Correlating images 18 and 26
Correlating images 19 and 20
Correlating images 19 and 21
Correlating images 19 and 22
Correlating images 19 and 23
Correlating images 19 and 24
Correlating images 19 and 25
Correlating images 19 and 26
Correlating images 20 and 21
Correlating im

### Identify outliers in shift matrix
Show the shift matrix with show)Rij() method.
If applicable, set range of minimum and maximum images to include with set_nz(nz_min,nz_max) method.
Find outliers with get_outliers(method, *args) method.  method parameter can be "NN", or "transitivity".  "NN" identifies outliers using only nearest neighbor elements of shift matrix, and requires an additional threshold parameter.  "transitivity" identifies outliers by identifying matrix elements which are inconsistent with physical stage positions, which must obey additive transitivity, and requires an additional threshold parameter, and an optional num_paths parameter.

In [13]:
# Show Xij and Yij matrices
s.show_Rij()

In [18]:
# Create mask, defining unuseable data points

s.set_nz(0,27)               # Set min/maz useable image indices
s.get_outliers("transitivity",10,10)       # Set outlier threshhold
s.set_bad_images([])         # Set bad images

s.show_Rij(mask=True)

In [19]:
# Optionally, add or remove points from the outlier mask that were not correctly identified

correct_pairs=[[11,12],[11,13]]
incorrect_pairs=[[1,9]]

for pair in correct_pairs:
    i,j=pair[0],pair[1]
    s.Rij_mask[i,j]=1
    s.Rij_mask[j,i]=1
for pair in incorrect_pairs:
    i,j=pair[0],pair[1]
    s.Rij_mask[i,j]=0
    s.Rij_mask[j,i]=0
    
s.show_Rij()

In [20]:
s.Rij_mask[:5,15]=0
s.Rij_mask[:3,18]=0
s.Rij_mask[:8,24:26]=0
s.Rij_mask[15,:5]=0
s.Rij_mask[18,:3]=0
s.Rij_mask[24:26,:8]=0

s.show_Rij()

### Remove outliers
The make_corrected_Rij() method determines the correct values for outliers in the shift matrices using transitivity, and the show_Rij_c() method shows the corrected shift matrices.

This step is optional, as it is performed automatically when get_average_image() is run if not called manually.
However, manually calling make_corrected_Rij() is recommended so that the final shift matrices can be visually inspected to ensure physical consistency.

In [21]:
s.make_corrected_Rij()
s.show_Rij_c()

### Calculate average image

In [22]:
# Create registered image stack and average

s.get_averaged_image()

In [23]:
# Display final image

s.show()

In [24]:
# Display report of registration procedure

s.show_report()

In [25]:
# Save report of registration procedure

s.save_report("/Users/Ben/Desktop/test_report.pdf")

Note that the full field of view of the original data has been preserved in the final image for computational simplicity, however, data at the edges is not physically meaningful.  All shifts are stored as 1D arrays in the attributes shifts_x and shifts_y, thus to obtain the limits of physical meaningful data, use, e.g. s.shifts_x.max(), s.shifts_x.min(), etc.